# MicroTVM Tutorial (ESP-IDF)

While functional verification using actual samples from the dataset is a useful first step, the main goal is to use the generated kernels on a real embedded device to run real-time inference using a microphone.

**MicroTVM** allows deploying TVM generated models to a micro-controller in a straightforward fashion. The following give an overview of the required steps to deploy out model on the ESP32-C3 development board.

## Project API

### Template Projects

MicroTVM comes with some template projects to get started i.e. for the Zephyr or Arduino platform. For this lab we provide a 3rd part Project API template using the ESP-IDF toolchain which can be found in `3_run/microtvm/template_project`. See `3_run/microtvm/README.md` for more information.

Two different types of ESP-IDF specific projects are supported:

- `micro_kws`: The standalone MicroKWS target software including all driver code and student tasks. (**Use this one for the lab exercises!**)
- `host_driven`: Allows TVM to communicate with the microcontroller via a RPC connection for debugging, auto-tuning,... (**optional**, see below for more information)

### MicroTVM

The following introduces the basic usage of the MicroTVM command line interface (based on TVMC which you have already used in lab 1).

The MicroTVM command line can be invoked via `tvmc micro [create-project|build|flash]`.

#### 1. Initializing a MicroTVM project

Untuned:

In [ ]:
!tvmc micro create prj ../2_deploy/gen/mlf.tar template --template-dir microtvm/template_project \
    --project-option project_type=micro_kws idf_target=esp32c3 num_classes=10

*Hint:** To overwrite an existing directory, you can use `tvmc micro create -f` instead. Just make sure to NOT deleta any of you local changes!

Tuned:

**Task**: Using of the previous example, figure out, how to generate `3_target/prj_tuned` based on the `2_deploy/gen/mlf_tuned.tar` artifact.

#### 2. Compiling the generated project

Untuned:

In [ ]:
!tvmc micro build prj template --template-dir microtvm/template_project \
    --project-option verbose=true idf_target=esp32c3

Tuned:

**Task**: Run the previous command for the `prj_tuned` project as well!

#### 3. Flashing the compiled program

Untuned:

In [ ]:
!tvmc micro flash prj template --template-dir microtvm/template_project \
    --project-option idf_target=esp32c3

Tuned:

**Task**: Run the previous command for the `prj_tuned` project as well!

#### 4. Modifying the provided MicroKWs code

While the template code is provided in `3_run/microtvm/template_project/src/micro_kws`, you should make your changes only in the generated `prj_tuned` directory. Please make sure to not overwrite your changes by removing or overwriting `prj_tuned`.

Doing this, it might be more straightforward to start using `idf.py build` and `idf.py flash monitor` from inside the `3_run/prj_tuned` directory instead of using the `micro [build|flash]` commands all the time.

#### 4. Testing the flashed model

In [ ]:
!tvmc run gen/prj --device micro

#### 5. Optional: Host Driven execution

**WORK IN PROGRESS**

The following part of the tutorial is following a different approach compared to the standalone MicroKWS application. The `host_driven` project type does only include the inference code and MicroTVM RPC interface for communicating with the host system.

Why is this useful? This enables debugging as well as benchmarking our model on-device in a separated environment. Unfortunately the previously generated AoT MLF artifacts can not be used together with the `host_driven` project type as this point in time. The rest of this tutorial therfore generates a new MLF using TVMs Graph runtime instead of the Ahead-of-Time (AoT) executor. As this might be confusing, please only contnue with the tutorial if you are really insterested in learning more about MicroTVM.



##### 5.1 Generating new model artifacts

In [ ]:
!TODO

##### 5.2 Setup up a new project

In [ ]:
!tvmc micro create -f prj_rpc ../2_deploy/gen/mlf_graph.tar template --template-dir microtvm/template_project \
    --project-option project_type=host_driven idf_target=esp32c3

##### 5.3 Compiling & Flashing the program

In [ ]:
!tvmc micro build -f prj_rpc template --template-dir microtvm/template_project \
    --project-option verbose=False idf_path=foo idf_tools_path=bar

In [ ]:
!tvmc micro flash -f prj_rpc template --template-dir microtvm/template_project

##### 5.4 Testing the model on-device

In [ ]:
!TODO

##### 5.5 On-device profiling of the model

This step has to be performed in Python because the TVMC command line currently does not allow profiling a MicroTVM model on-device. The following code is aslso available in he `profile.py` script which can be found in this directory.

In [ ]:
import os
import tvm.micro


def profile_micro(mlf_dir: str, prj_dir: str):
    project = tvm.micro.GeneratedProject.from_directory(prj_dir)
    graph_json = open(os.path.join(mlf_dir, ""))
    with tvm.micro.Session(project.transport()) as session:
        debug_module = tvm.micro.create_local_debug_executor(
            lowered_tuned.get_graph_json(), session.get_system_lib(), session.device
        )
        debug_module.set_input(**lowered_tuned.get_params())
        debug_module.run()
        del debug_module